# Homework 2: Denoising Auto-Encoders with TensorFlow
--------- 
Nikolas Wolfe<br>
11-785 Deep Learning Seminar<br>
December 20th, 2015
<br>
<h2 style="color:red;">Note:</h2><br>
I used Google <a href="https://www.tensorflow.org/">TensorFlow</a> for this homework. TensorFlow is a new, cutting-edge ML/Deep Learning library released in November 2015. It's claimed advantages over existing deep learning frameworks (some of which I can verify are true) are its ability to seamlessly handle the optimization of computations using an efficient back-end written in C++ and run on a CPU, GPU, or multiple distributed GPUs using the same code base. In their <a href='http://download.tensorflow.org/paper/whitepaper2015.pdf'>whitepaper</a>, Google claims the purpose of TensorFlow is to do large-scale machine learning on heterogeneous distributed systems.

As promising as these claims sound, TensorFlow is definitively <em>not</em> ready for prime-time. The <a href="https://www.tensorflow.org/versions/master/api_docs/index.html">API</a> is terse and incomplete, and there are few documented implementations of systems beyond the canned examples on the <a href="https://www.tensorflow.org/versions/master/tutorials/index.html">website</a>. Doing this homework required building new infrastructure for not only constructing arbitrarily designed networks, e.g. a stacked denoising auto-encoder (which has already been <a href="https://github.com/hussius/ascii-autoencoder/blob/master/ascii_autoencoder.py">tried</a> by some independent parties), but also reading in an <a href="http://host.robots.ox.ac.uk/pascal/VOC/voc2007/index.html">external</a> dataset. These are all tasks which have yet to be documented in any detail by Google, and TensorFlow is critically lacking in this respect. The documentation deals exlusively with very clean and expediently formatted datasets. 

In summary, I cannot claim to have gotten this homework working entirely, though the process has been spec'd out in extensive detail here and maybe in the near future someone will find a way to make this work. Rest assured that it <em>can</em> work, but the API is still too limited for something like this.

<h3>TensorFlow is good for 11-785!</h3>

This is not to completely disregard the potential of TensorFlow. Some of the <a href="https://www.tensorflow.org/versions/master/tutorials/index.html">tutorials</a> are actually quite substantial, including examples of convolutional networks for image classification on the <a href="https://www.tensorflow.org/versions/master/tutorials/deep_cnn/index.html">CIFAR-10</a> and <a href="https://www.tensorflow.org/versions/master/tutorials/image_recognition/index.html">ImageNet</a> datasets, <a href="https://www.tensorflow.org/versions/master/tutorials/word2vec/index.html">word2vec</a>, <a href="https://www.tensorflow.org/versions/master/tutorials/recurrent/index.html">RNNs & LSTMs</a> and how to do neural network based <a href="https://www.tensorflow.org/versions/master/tutorials/seq2seq/index.html">Machine Translation</a>, among other things. It is for this reason that in the future I would <em>highly</em> recommend the use of TensorFlow for the pedagogical purposes of 11-785. On the website, the above topics were <a href="http://deeplearning.cs.cmu.edu/labs/labs.html">all</a> listed as things which were originally going to be made into homeworks, and thus this frameowork provides a useful foundation to get the more lay students going with deep learning. TensorFlow is likely to be well-supported into the future, it will get faster, and many observers ([<a href="http://www.infoworld.com/article/3003920/data-science/4-no-bull-takeaways-about-googles-machine-learning-project.html">1</a>][<a href="http://www.pcmag.com/article2/0,2817,2494727,00.asp">2</a>][<a href="http://www.geekwire.com/2015/google-open-sources-tensorflow-machine-learning-system-offering-its-neural-network-to-outside-developers/">3</a>][<a href="http://www.wired.com/2015/11/google-open-sources-its-artificial-intelligence-engine/">4</a>]) agree it has a fundamentally more flexibile (if not intuitive) design than Theano, Torch or Caffe, and its rough-around-the-edges bleeding-edge state arguably makes it ideal for exploration in a class like this. 

## Problem 1
Download PASCAL VOC2007 dataset from VOC2007 website. This dataset includes training, validation and testing sets. You can fnd the images inside JPEGImages folder and the labels inside the ImageSets/Main folder.

I downloaded the VOC2007 dataset, and I have placed the train/test data in two folders:

In [ ]:
%%bash
ls -l train/VOC2007/
ls -l test/VOC2007/

Together there are 2501 training images, 2510 validation images and 4952 testing images. Train a Denosing Autoencoder with different noise levels using the training set and visualize the learned filters as in here. Here you can use the whole image (re-size to a fixed size, e.g., 256x256) or sub-sampled small patches with a fixed size (e.g., 30x30). Comparing both of them will give you extra credits.
### Image Input Types to Test:
* 256 x 256 Resizing
* Convolutional sub-samples (30 x 30)

### Imports

In [ ]:
# ------- TENSORFLOW --------- #
import tensorflow as tf
import sys, os, math, numpy

from collections import defaultdict
from collections import OrderedDict
from sklearn import svm

### Constants

In [ ]:
# ------------------ General ------------------- #
IMG_DIR = 'VOC2007' + os.sep + 'JPEGImages' + os.sep
LBL_DIR = 'VOC2007' + os.sep + 'ImageSets' + os.sep + 'Main' + os.sep

# ------------------ Training ------------------ #
TRAIN_DIR = 'train' + os.sep + IMG_DIR
TRAIN_LABELS = 'train' + os.sep + LBL_DIR

# ------------------ Testing ------------------- # 
TEST_DIR = 'test' + os.sep + IMG_DIR
TEST_LABELS = 'test' + os.sep + LBL_DIR

# ------------------ Misc ------------------- # 
NUM_CLASSES = 20
ACTIVATION = 'tanh'

In [ ]:
print(TRAIN_DIR)
print(TRAIN_LABELS)
print(TEST_DIR)
print(TEST_LABELS)

The labels for the images are stored in the 'Main' directory, like so:

In [ ]:
%%bash
ls train/VOC2007/ImageSets/Main/ | head -4

The data in these files are in the following form, where the 1st column corresponds to the jpeg file and the 2nd column corresponds to whether the label is present, as shown in the output below:

In [ ]:
%%bash
tail train/VOC2007/ImageSets/Main/bicycle_train.txt | head -4

Just to verify this, we can look at somet of these files from the 'JPEGImages' folder:

In [ ]:
from IPython.display import Image
from IPython.display import display
x = Image(filename=TRAIN_DIR + '009926.jpg', width=256, height=256) # this IS a bicycle
y = Image(filename=TRAIN_DIR + '009940.jpg', width=256, height=256) # this is NOT a bicycle
display(x, y)

## Handling the Data
We first need to import the dataset and get it into the appropriate data structures which are used in TensorFlow. In this case I will not separate the training and validation sets.

In [ ]:
# Generic read data from our file structure...
def get_data(DIR, LABELS, key):
    imgs = {}
    labels = defaultdict(list)
    label_list = []
    for root, dirs, files in os.walk(LABELS, topdown=False):
        for f in files: 
            if '_' + key in f: 
                label = f.split('_')[0] 
                label_list.append(label)
                f = os.path.abspath(LABELS + f)
                pos_ex = [l.split()[0] for l in open(f).readlines() if l.split()[-1] == '1']
                for img in pos_ex:
                    labels[label].append(img)
                    imgs[img] = os.path.abspath(DIR + img + '.jpg')
    return imgs, labels, label_list

# Dig through for the training data...
def get_train_data(): return get_data(TRAIN_DIR, TRAIN_LABELS, "trainval.txt")        

# Dig through for the testing data...
def get_test_data(): return get_data(TEST_DIR, TEST_LABELS, "test.txt")        

Read in train and test data...

In [ ]:
# --------- TRAIN & TEST SETS --------- #
train_imgs, train_labels, label_list = get_train_data()
test_imgs, test_labels, label_list = get_test_data()
print('Labels: ' + ', '.join(label_list))

For use in classification later on, we're going to find it useful to have 20-dimensional 1-hot bit vectors to represent our 20 PASCAL classification labels. So we'll go ahead and create those representations here:

In [ ]:
def get_one_hot_label_list(label_dict):
    keys = OrderedDict(sorted(label_dict.items())).keys()
    one_hot_lookup = {}
    for i in range(len(keys)):
        one_hot = numpy.zeros(NUM_CLASSES)
        one_hot[i] = 1
        one_hot_lookup[keys[i]] = one_hot
    one_hot_list = []
    for label in label_dict.keys():
        vals = label_dict[label]
        for v in vals:
            one_hot_list.append((v, one_hot_lookup[label]))
    return one_hot_list

This will create a mapping between the image keys and the one-hot labels. We add a duplicate image key entry to the list if the image contains more than one label.

In [ ]:
train_labels_one_hot = get_one_hot_label_list(train_labels)
test_labels_one_hot = get_one_hot_label_list(test_labels)

Now we need to read in the images and create TensorFlow tensors with them... In this function we will associate the one-hot dictionaries with the files their keys point to. As per the requirements of the first part of this assignment, we will also call an operation to resize the images to 256 x 256 pixels.

In [ ]:
# Reads in data, creates grey-scale img...
def read_img_data(one_hot_dict, file_dict):
    labels = []
    files = []
    img_files = []
    reader = tf.WholeFileReader()
    print('Assembling files...')
    for (img, label_vec) in one_hot_dict:
        files.append(file_dict[img])
        labels.append(label_vec)
    queue = tf.train.string_input_producer(files)
    jpg_key, jpg_img = reader.read(queue)
    # grey-scale the image...
    jpg_img = tf.image.decode_jpeg(jpg_img, channels=1)
    init = tf.initialize_all_variables()
    # Run session...
    print('Starting tensorflow session...')
    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            sess.run(init)
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)
            for i in range(len(files)):
                jpg = jpg_img.eval()
                jpg = numpy.asarray(jpg)
                jpg.resize((256, 256))
                img_files.append(jpg)
            coord.request_stop()
            coord.join(threads)
            
    # build numpy arrays
    print('Reconstructing arrays...')
    img_files = numpy.asarray(img_files)
    labels = numpy.asarray(labels)
    print('Done!')
    return img_files, labels

### Get Data By Class Label

In [ ]:
def get_train_data_for_label(label):
    print('Getting data for ' + label + '...')
    imgs_for_label = train_labels[label]
    lbl_dict = {}
    lbl_dict[label] = imgs_for_label
    one_hot_dict = get_one_hot_label_list(lbl_dict)
    img_files, labels = read_img_data(one_hot_dict, train_imgs)
    print("Tensor shape: " + str(numpy.shape(img_files)))
    return img_files, labels
    
def get_test_data_for_label(label): 
    print('Getting data for ' + label + '...')
    imgs_for_label = test_labels[label]
    lbl_dict = {}
    lbl_dict[label] = imgs_for_label
    one_hot_dict = get_one_hot_label_list(lbl_dict)
    img_files, labels = read_img_data(one_hot_dict, test_imgs)
    print("Tensor shape: " + str(numpy.shape(img_files)))
    return img_files, labels

### Example Use:

In [ ]:
img_files, labels = get_test_data_for_label('bird')
img_files, labels = get_train_data_for_label('sofa')

# Building Auto-Encoders
An auto-encoder can be defined as follows. For $k$ dimensional data, we reduce the data to a lower dimensional representation $j$ using a matrix multiplication:

\begin{align}
softmax\left(W * x + b\right) &= x^{\prime}
\end{align}

Where $W$ is a matrix from $R^k \rightarrow R^j$ and $x^{\prime}$ is the lower dimensional representation of the input. 

The reconstruction function maps the matrix $W^{\prime}$ from $R^j$ to $R^k$:

\begin{align}
softmax\left(W^{\prime} * x^{\prime} + b^{\prime}\right)
\end{align}

The function we want to minimize overall is:

\begin{align}
cost = || \ softmax^{\prime} \left(W^{\prime} * (softmax(W * x + b)) + b^{\prime}\right) - x \ ||
\end{align}

This is essentially minimizing the cross-entropy of the reconstruction with the original input. And for a deep autoencoder, as in Problem 2, we simply stack successive layers of these reductions. The code below will work for both stacked and non-stacked encoders. First, we need to be able to construct an auto-encoder given a shape description...

In [ ]:
# ------------------------------------------------------------- #
# Returns a weight tensor with the specified dimensions
# ------------------------------------------------------------- #
def get_weight_tensor(input_dim, output_dim):
    shape = [input_dim, output_dim]
    std = 1.0 / math.sqrt(float(input_dim))
    rand_init_values = tf.truncated_normal(shape, stddev=std)
    return tf.Variable(rand_init_values)

# ------------------------------------------------------------- #
# Returns a bias tensor with the specified dimension
# ------------------------------------------------------------- #
def get_bias_tensor(dim):
    zero_init_values = tf.zeros([dim], tf.float32)
    return tf.Variable(zero_init_values)

# ------------------------------------------------------------- #
# Get the weighted sum of the inputs plus the bias
# ------------------------------------------------------------- #
def get_mult_op(inputs, weights, bias):
    return tf.matmul(inputs, weights) + bias

# ------------------------------------------------------------- #
# Applies the parameterized activation function to the layer
# ------------------------------------------------------------- #
def get_activation(layer, type='sigmoid'):
    if type is 'sigmoid': return tf.nn.sigmoid(layer)
    elif type is 'relu': return tf.nn.relu(layer)
    elif type is 'softmax': return tf.nn.softmax(layer)
    elif type is 'tanh': return tf.nn.tanh(layer)
    elif type is 'linear': return layer
    else: return layer
    
# ------------------------------------------------------------- #
# Builds an autoencoder structure for the input vector and shape
# ------------------------------------------------------------- #
def build_autoencoder_for_shape(input_vector, shape):
    # ----------------------------------------------------------- #
    # Layer input
    layer_input = input_vector
    # ----------------------------------------------------------- #
    # Weight matrices for encoder structure
    encoder_weights = []
    # ----------------------------------------------------------- #
    # Build the layers based on the vector dimensions in shape...
    # ----------------------------------------------------------- #
    print("Building layer... | layer dim: " + str(int(layer_input.get_shape()[1])))
    for layer_dimension in shape:
        # ------------------------------------------------------------- #
        # Get the layer dimensions...
        print("Building layer... | layer dim: " + str(layer_dimension))
        # ------------------------------------------------------------- #
        # Build a weight matrix...
        input_dimension = int(layer_input.get_shape()[1])
        W = get_weight_tensor(input_dimension, layer_dimension)
        # ------------------------------------------------------------- #
        # Build a bias vector...
        b = get_bias_tensor(layer_dimension)
        # ------------------------------------------------------------- #
        # Append the weight matrix to the encoder data structure...
        encoder_weights.append(W)
        # ------------------------------------------------------------- #
        # Layer input multiplication
        X = get_mult_op(layer_input, W, b)
        # ------------------------------------------------------------- #
        # Get the layer output computation, e.g. O = sigmoid(X)
        O = get_activation(X, type=ACTIVATION)
        # ------------------------------------------------------------- #
        # Plug the output of this layer to the input of the next...
        layer_input = O
        
    # ------------------------------------------------------------- #
    # Reached the middle layer encoding of the input... 
    input_encoding = layer_input
    
    # ------------------------------------------------------------- #
    # Now work your way out with symmetric structure about the middle
    shape.reverse()
    encoder_weights.reverse()
    
    # ------------------------------------------------------------- #
    # Stacked Autoencoder layers + output dimension...
    # Skip (reversed) shape[0] b/c that's the encoder layer!
    shape = shape[1:] + [int(input_vector.get_shape()[1])]
    
    # ------------------------------------------------------------- #
    # For tied weights, we can just use the encoder_weights[i] and
    # transpose them!
    for i, layer_dimension in enumerate(shape):
        # ------------------------------------------------------------- #
        print("Building layer... | layer dim: " + str(layer_dimension))
        # ------------------------------------------------------------- #
        # Transpose the weight matrix...
        W = tf.transpose(encoder_weights[i])
        # ------------------------------------------------------------- #
        # Bias vector...
        b = get_bias_tensor(layer_dimension)
        # ------------------------------------------------------------- #
        # Input operation... Also have to transpose the input!
        X = get_mult_op(tf.transpose(layer_input), W, b)
        # ------------------------------------------------------------- #
        # Activation operation, e.g. O = sigmoid(X)
        O = get_activation(X, type=ACTIVATION)
        # ------------------------------------------------------------- #
        # Plug the output of this layer to the input of the next...
        # Also have to transpose here...
        layer_input = tf.transpose(O)
        
    # ------------------------------------------------------------- #
    # Autoencoder output: Reconstruction of the input... (transpose)
    input_reconstruction = tf.transpose(layer_input)
    
    # ------------------------------------------------------------- #
    # MINIMIZE THE CROSS ENTROPY BETWEEN THE INPUT & RECONSTRUCTION
    # ------------------------------------------------------------- #
    cost_function = tf.sqrt(tf.reduce_mean(tf.square(input_vector - input_reconstruction)))
    # ------------------------------------------------------------- #
    
    # ------------------------------------------------------------- #
    # Return hash structure with encoder, decoder, and cost func
    # ------------------------------------------------------------- #
    return {
        'encoder' : input_encoding,
        'decoder' : input_reconstruction,
        'cost'    : cost_function
    }

### Example: Single Layer Auto-Encoder

In [ ]:
def build_single_layer_autoencoder(input_dim, reduced_dim):
    x = tf.placeholder("float", [None, input_dim])
    autoencoder = build_autoencoder_for_shape(x, [reduced_dim])
    return autoencoder

Run it with TensorFlow:

In [ ]:
input_dim = 25
reduced_dim = 5

In [ ]:
with tf.Session() as sess:
    autoencoder = build_single_layer_autoencoder(input_dim, reduced_dim)
    init = tf.initialize_all_variables()
    sess.run(init)

We'll move on to <strong>Problem 2</strong> just for convenience sake now, then talk about the feature extraction...

-------

## Problem 2
Given the PASCAL dataset from Part 1, train a Stacked Denosing Autoencoder
as in here using the training set and use it as:

* A fixed network to extract features. Train RBF SVMs using the extracted features from the training set. Again, here you can use a small network trained with sub-sampled inputs and average the output of the network to get image-level features when you train the SVMs.

* A pre-trained network to initialize a feed-forward network and finne-tune the feed-forward network using the training set. Here you will have to use the whole re-sized images as the inputs. Given the fact that an image may have multiple labels in this dataset, you may not want to use a softmax layer as your output layer. Explore the network structure and SVM parameters to achieve the best validation accuracy (mAP for SVM) you can get. Report the best network structure, SVM parameters the corresponding testing MAP.

### Example: Stacked Deep Auto-Encoder

In [ ]:
def build_stacked_autoencoder(input_dim, shape):
    x = tf.placeholder("float", [None, input_dim])
    autoencoder = build_autoencoder_for_shape(x, shape)
    return autoencoder

Run it with TensorFlow:

In [ ]:
input_dim = 1024
shape = [512, 256, 128, 64, 32, 16, 8, 4, 2]

In [ ]:
with tf.Session() as sess:
    autoencoder = build_stacked_autoencoder(input_dim, shape)
    init = tf.initialize_all_variables()
    sess.run(init)

Arguably a bit overkill... but you get the idea!

## Running & Feature Extraction

## Training an RBF-Kernel SVM Classifier
Below is a skeleton class for training an RBF-Kernel SVM classifier. <strong>NOTE:</strong> This is largely copied from the SciLearn documentation: <a href="http://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html#example-svm-plot-rbf-parameters-py">Click here!</a>

First, some imports from SciLearn's LibSVM wrapper:

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.grid_search import GridSearchCV

Utility function to move the midpoint of a colormap to be around the values of interest:

In [5]:
class MidpointNormalize(Normalize):

    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

### Train and Run SVM

In [8]:
##############################################################################
# Load and prepare data set
#
# dataset for grid search
def run_rbf_svm():
    iris = load_iris()
    X = iris.data
    y = iris.target

    # Dataset for decision function visualization: we only keep the first two
    # features in X and sub-sample the dataset to keep only 2 classes and
    # make it a binary classification problem.

    X_2d = X[:, :2]
    X_2d = X_2d[y > 0]
    y_2d = y[y > 0]
    y_2d -= 1

    # It is usually a good idea to scale the data for SVM training.
    # We are cheating a bit in this example in scaling all of the data,
    # instead of fitting the transformation on the training set and
    # just applying it on the test set.

    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X_2d = scaler.fit_transform(X_2d)

    ##############################################################################
    # Train classifiers
    #
    # For an initial search, a logarithmic grid with basis
    # 10 is often helpful. Using a basis of 2, a finer
    # tuning can be achieved but at a much higher cost.

    C_range = np.logspace(-2, 10, 13)
    gamma_range = np.logspace(-9, 3, 13)
    param_grid = dict(gamma=gamma_range, C=C_range)
    cv = StratifiedShuffleSplit(y, n_iter=5, test_size=0.2, random_state=42)
    grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
    grid.fit(X, y)

    print("The best parameters are %s with a score of %0.2f"
          % (grid.best_params_, grid.best_score_))

    # Now we need to fit a classifier for all parameters in the 2d version
    # (we use a smaller set of parameters here because it takes a while to train)

    C_2d_range = [1e-2, 1, 1e2]
    gamma_2d_range = [1e-1, 1, 1e1]
    classifiers = []
    for C in C_2d_range:
        for gamma in gamma_2d_range:
            clf = SVC(C=C, gamma=gamma)
            clf.fit(X_2d, y_2d)
            classifiers.append((C, gamma, clf))

    ##############################################################################
    # visualization
    #
    # draw visualization of parameter effects

    plt.figure(figsize=(8, 6))
    xx, yy = np.meshgrid(np.linspace(-3, 3, 200), np.linspace(-3, 3, 200))
    for (k, (C, gamma, clf)) in enumerate(classifiers):
        # evaluate decision function in a grid
        Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)

        # visualize decision function for these parameters
        plt.subplot(len(C_2d_range), len(gamma_2d_range), k + 1)
        plt.title("gamma=10^%d, C=10^%d" % (np.log10(gamma), np.log10(C)),
                  size='medium')

        # visualize parameter's effect on decision function
        plt.pcolormesh(xx, yy, -Z, cmap=plt.cm.RdBu)
        plt.scatter(X_2d[:, 0], X_2d[:, 1], c=y_2d, cmap=plt.cm.RdBu_r)
        plt.xticks(())
        plt.yticks(())
        plt.axis('tight')

    # plot the scores of the grid
    # grid_scores_ contains parameter settings and scores
    # We extract just the scores
    scores = [x[1] for x in grid.grid_scores_]
    scores = np.array(scores).reshape(len(C_range), len(gamma_range))

    # Draw heatmap of the validation accuracy as a function of gamma and C
    #
    # The score are encoded as colors with the hot colormap which varies from dark
    # red to bright yellow. As the most interesting scores are all located in the
    # 0.92 to 0.97 range we use a custom normalizer to set the mid-point to 0.92 so
    # as to make it easier to visualize the small variations of score values in the
    # interesting range while not brutally collapsing all the low score values to
    # the same color.

    plt.figure(figsize=(8, 6))
    plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
    plt.imshow(scores, interpolation='nearest', cmap=plt.cm.hot,
               norm=MidpointNormalize(vmin=0.2, midpoint=0.92))
    plt.xlabel('gamma')
    plt.ylabel('C')
    plt.colorbar()
    plt.xticks(np.arange(len(gamma_range)), gamma_range, rotation=45)
    plt.yticks(np.arange(len(C_range)), C_range)
    plt.title('Validation accuracy')
    plt.show()

Run it!

In [ ]:
run_rbf_svm()